**Project Name** : Optimization of Supply Moves (OSM) <br/>
**Nestle SPOCs** :Aris Anadilla , Emily Knaus <br/>
**Created By** : Akansha Rana <br/>
**Purpose** : To Identify the Inventory that is on Risk at a DC and Recommend STOs to other DCs having demand to consume this AtRisk Inventory, to reduce the wastage <br/>
**Date** : <br/>
**Functionality** : This Notebook runs an optimization model based on scipy library that recommend inventory movement between two locations<br/>

**Output Folder** : 
1. solutions/ift/ift/outbount/Test


**Dependent Notebooks** : 1. Azure_Connection, 2. InputCreation_PowerApps, 3. Model_Run_Notebook_PowerApps </br>

**Widgets Definition**: </br>
<small>
1.partition_path:partition_path will tell about the part no of the data that needs to sent throught optimizer(integer)</br>
2.increment_value:increment value for each run(in each iteration the partition value will be increaed by this value (integer)</br>
3.destination_path:path of folder where the data should be saved</br>
4.dateformat: date in YYYY/MM/DD format(DataType = string)</br>

In [0]:
%run "./Azure_Connection"

In [0]:
# importing required libraries and functions
import numpy as np
import pandas as pd
from pyspark.sql.types import DateType
import pyspark.sql.functions as sf
from pyspark.sql.functions import current_date
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession
from pyspark.sql import Window
from datetime import date,datetime,timedelta
from sklearn.preprocessing import OneHotEncoder
from scipy.optimize import linprog

In [0]:
# Creatin parameters for the variable whose value will be passed from another notebook
# partition_path will tell about the part no of the data that needs to sent throught optimizer
dbutils.widgets.text("partition_path", "", "")
partition_path_value = dbutils.widgets.get("partition_path")

# increment value for each run
dbutils.widgets.text("increment_value", "", "")
increment_value = dbutils.widgets.get("increment_value")
increment_value = int(increment_value)

# destination value for the output
dbutils.widgets.text("destination_path", "", "")
destination_path = dbutils.widgets.get("destination_path")

# dateformat variable, which tells about the Date
dbutils.widgets.text("dateformat", "", "")
dateformat = dbutils.widgets.get("dateformat")
print(dateformat)
ModelRunDate = datetime.strptime(dateformat, "%Y/%m/%d").date()  # .strftime("%Y-%m-%d")
print(partition_path_value, increment_value, destination_path, dateformat, ModelRunDate)

#Reading Data files

In [0]:
# Reading the AllPossibleCombinations Data From Datalake
try:
    AllComb = spark.read.format("delta").load(
        destination_path + "AllPossibleCombinations"
    )
except Exception as e:
    print(f"Error in loading AllPossibleCombinations data: {str(e)}")
    raise SystemExit(f"Exiting due to the error: {str(e)}")
AllComb = AllComb.filter(sf.col("Report_Run_Date") == ModelRunDate)

In [0]:
# here all the data is distributed in to multiple parts based on the materials. Materials are sorted in the order of their count of rows, and material count falling in a specific range is run in one loop
sku_counts = AllComb.groupby("MaterialID").count()
sku_counts = sku_counts.withColumn(
    "cum_sum",
    sf.sum("count").over(
        Window.orderBy("count", "MaterialID").rowsBetween(Window.unboundedPreceding, 0)
    ),
)
# filtering material for the specific range of count based on the partition value
sku_counts = sku_counts.filter(
    (sf.col("cum_sum") <= (int(partition_path_value) + increment_value))
    & (sf.col("cum_sum") > int(partition_path_value))
)

In [0]:
# selecting the material for inscope based on sku_counts
AllComb = AllComb.join(sku_counts.select("MaterialID"), ["MaterialID"])

# if there is no material present in any specific range, there is no need to run the notebook further, so exiting
row_count = AllComb.count()
if row_count <= 0:
    dbutils.notebook.exit("No Material Present")

In [0]:
# converting the required columns to a pandas dataframe
AllComb_pd_cpy = AllComb.select(
    "MaterialID",
    "Source",
    "SalvageDate",
    "QuantityinUoM",
    "Destination",
    "Date",
    "ExcessDemand",
    "Min_Of_Upcoming_Days",
    "arrivaldate",
    "IntermediateDemandColumn",
    "ActualExcessDemand",
    "source_dest_sku_salvagedate",
    "source_sku_salvagedate",
    "source_sku_salvagedate_date",
    "source_dest_sku_salvagedate_arrivaldate",
    "source_dest_arrivaldate",
    "source_dest_dispatchDate",
    "dest_sku",
    "dest_sku_date",
    "dest_sku_date_arrivaldate",
    "OBJ",
).toPandas()

In [0]:
# list of all distinct combinations
source_dest_sku_salvagedate_arrivaldate_list = AllComb_pd_cpy[
    "source_dest_sku_salvagedate_arrivaldate"
].tolist()
len(source_dest_sku_salvagedate_arrivaldate_list)

In [0]:
# performing onHotEncoding and creatinf a dummy Identity matrix that will tell about the selection of any combination for the recommendation
AllComb_pd_cpy["srno"] = AllComb_pd_cpy.index + 1
AllComb_pd_cpy["min_inv"] = AllComb_pd_cpy[["QuantityinUoM", "ActualExcessDemand"]].min(
    axis=1
)
rated_dummies = pd.get_dummies(AllComb_pd_cpy.srno)
AllComb_pd_cpy = pd.concat([AllComb_pd_cpy, rated_dummies], axis=1)

In [0]:
column_names = [
    i for i in range(1, len(source_dest_sku_salvagedate_arrivaldate_list) + 1)
]

#defining objective function
# the objective function is to maximize AllComb_pd_cpy['OBJ'], but with scipy it can only be minimized ,so we will minimize AllComb_pd_cpy['OBJ'] * -1 and get our work done
obj = np.array(AllComb_pd_cpy["OBJ"] * -1)
lhs = np.array(AllComb_pd_cpy[column_names])
qty_movable_source_dest_sku_salvagedate_arrivaldate = AllComb_pd_cpy["min_inv"]
rhs = np.array(AllComb_pd_cpy["min_inv"])

In [0]:
# first part of constraint. source_sku_salvahedate level moved qty should be less than the available qty
source_sku_salvagedate_movedfrom = AllComb_pd_cpy.groupby("source_sku_salvagedate").agg(
    {"QuantityinUoM": "mean"}
)
source_sku_salvagedate_movedfrom1 = AllComb_pd_cpy.groupby("source_sku_salvagedate")[
    column_names
].apply(lambda x: x.astype(int).sum())
source_sku_salvagedate_movedfrom = source_sku_salvagedate_movedfrom.join(
    source_sku_salvagedate_movedfrom1, on="source_sku_salvagedate"
)

In [0]:
# Adding constraint in the model
lhs1 = np.concatenate(
    (lhs, np.array(source_sku_salvagedate_movedfrom[column_names])), axis=0
)
rhs1 = np.concatenate(
    (rhs, np.array(source_sku_salvagedate_movedfrom["QuantityinUoM"])), axis=0
)

In [0]:
del source_sku_salvagedate_movedfrom, source_sku_salvagedate_movedfrom1, lhs, rhs

In [0]:
# second part of constraint. dest_sku_date level moved qty should be less than the usable demand
dest_sku_date_movedto1 = (
    AllComb_pd_cpy.sort_values(by="dest_sku_date_arrivaldate")
    .groupby(["Destination", "MaterialID", "Date"])
    .agg(
        {
            "ExcessDemand": "mean",
            "IntermediateDemandColumn": "mean",
            "Min_Of_Upcoming_Days": "mean",
        }
    )
)
dest_sku_date_movedto2 = (
    AllComb_pd_cpy.sort_values(by="dest_sku_date_arrivaldate")
    .groupby(["Destination", "MaterialID", "Date"])[column_names]
    .apply(lambda x: x.astype(int).sum())
)

dest_sku_date_movedto2 = (
    dest_sku_date_movedto2.sort_values(
        by=["Destination", "MaterialID", "Date"], ascending=True
    )
    .groupby(["Destination", "MaterialID"])[column_names]
    .apply(lambda x: x.cumsum())
)


dest_sku_date_movedto = dest_sku_date_movedto1.join(
    dest_sku_date_movedto2, on=["Destination", "MaterialID", "Date"]
).reset_index()

In [0]:
# Adding constraint in the model
lhs2 = np.concatenate((lhs1, np.array(dest_sku_date_movedto[column_names])), axis=0)
rhs2 = np.concatenate(
    (rhs1, np.array(dest_sku_date_movedto["Min_Of_Upcoming_Days"])), axis=0
)

In [0]:
del dest_sku_date_movedto, lhs1, rhs1

In [0]:
# 3rd part - second part of constraint. dest_sku_date_arrivaldate level moved qty should be less than the usable demand
dest_sku_date_arrival_movedto1 = (
    AllComb_pd_cpy.sort_values(by="dest_sku_date_arrivaldate", ascending=False)
    .groupby(["Destination", "MaterialID", "Date", "arrivaldate"])
    .agg({"ExcessDemand": "mean", "IntermediateDemandColumn": "mean"})
    .reset_index()
)
dest_sku_date_arrival_movedto1 = dest_sku_date_arrival_movedto1.sort_values(
    by=["Destination", "MaterialID", "Date", "arrivaldate"], ascending=False
)
dest_sku_date_arrival_movedto1["MovableQty"] = (
    dest_sku_date_arrival_movedto1["ExcessDemand"]
    - dest_sku_date_arrival_movedto1["IntermediateDemandColumn"]
)
dest_sku_date_arrival_movedto1.loc[
    dest_sku_date_arrival_movedto1["MovableQty"] <= 0, "MovableQty"
] = 0

dest_sku_date_arrival_movedto2 = (
    AllComb_pd_cpy.sort_values(by="dest_sku_date_arrivaldate", ascending=False)
    .groupby(["Destination", "MaterialID", "Date", "arrivaldate"])[column_names]
    .apply(lambda x: x.astype(int).sum())
)  # .reset_index()

dest_sku_date_arrival_movedto2 = (
    dest_sku_date_arrival_movedto2.sort_values(
        by=["Destination", "MaterialID", "Date", "arrivaldate"], ascending=False
    )
    .groupby(["Destination", "MaterialID", "Date"])[column_names]
    .apply(lambda x: x.cumsum())
)

In [0]:
# Adding constraint in the model
lhs3 = np.concatenate(
    (lhs2, np.array(dest_sku_date_arrival_movedto2[column_names])), axis=0
)
rhs3 = np.concatenate(
    (rhs2, np.array(dest_sku_date_arrival_movedto1["MovableQty"])), axis=0
)

In [0]:
del dest_sku_date_arrival_movedto1, dest_sku_date_arrival_movedto2, lhs2, rhs2

In [0]:
# 4th part when there are multiple batchs recommended at one destination for one material, at each date and arrival date combination sum of all previous recommended qty should not exceed the usable excess demand
All_ArrivalDates = AllComb_pd_cpy[
    ["Destination", "MaterialID", "arrivaldate"]
].drop_duplicates()
All_DemandDates = AllComb_pd_cpy[
    ["Destination", "MaterialID", "Date"]
].drop_duplicates()

All_DemandDates_ArrivalDates = All_DemandDates.merge(
    All_ArrivalDates, on=["Destination", "MaterialID"]
)
All_DemandDates_ArrivalDates = All_DemandDates_ArrivalDates[
    All_DemandDates_ArrivalDates["Date"] > All_DemandDates_ArrivalDates["arrivaldate"]
]

All_DemandDates_ArrivalDates = All_DemandDates_ArrivalDates.merge(
    AllComb_pd_cpy[["Destination", "MaterialID", "Date", "arrivaldate"] + column_names],
    on=["Destination", "MaterialID"],
)

All_DemandDates_ArrivalDates = All_DemandDates_ArrivalDates[
    All_DemandDates_ArrivalDates["Date_x"] >= All_DemandDates_ArrivalDates["Date_y"]
]

All_DemandDates_ArrivalDates = All_DemandDates_ArrivalDates[
    All_DemandDates_ArrivalDates["arrivaldate_x"]
    <= All_DemandDates_ArrivalDates["arrivaldate_y"]
]

All_DemandDates_ArrivalDates = (
    All_DemandDates_ArrivalDates.groupby(
        ["Destination", "MaterialID", "Date_x", "arrivaldate_x"]
    )[column_names]
    .apply(lambda x: x.sum())
    .reset_index()
)

Excess_demand_intermediate_column = AllComb_pd_cpy[
    [
        "Destination",
        "MaterialID",
        "Date",
        "arrivaldate",
        "Min_Of_Upcoming_Days",
        "IntermediateDemandColumn",
    ]
].drop_duplicates()
All_DemandDates_ArrivalDates.rename(
    columns={"Date_x": "Date", "arrivaldate_x": "arrivaldate"}, inplace=True
)

All_DemandDates_ArrivalDates = All_DemandDates_ArrivalDates.merge(
    Excess_demand_intermediate_column,
    on=["Destination", "MaterialID", "Date", "arrivaldate"],
)

All_DemandDates_ArrivalDates["MovableQty2"] = (
    All_DemandDates_ArrivalDates["Min_Of_Upcoming_Days"]
    - All_DemandDates_ArrivalDates["IntermediateDemandColumn"]
)
All_DemandDates_ArrivalDates.loc[
    All_DemandDates_ArrivalDates["MovableQty2"] <= 0, "MovableQty2"
] = 0

In [0]:
# Adding constraint in the model
lhs31 = np.concatenate(
    (lhs3, np.array(All_DemandDates_ArrivalDates[column_names])), axis=0
)
rhs31 = np.concatenate(
    (rhs3, np.array(All_DemandDates_ArrivalDates["MovableQty2"])), axis=0
)

In [0]:
del All_DemandDates_ArrivalDates, Excess_demand_intermediate_column, lhs3, rhs3

In [0]:
# 5th part- for any arrival date, qty recommended should not exceed the intermediate demand(between date and arrival date) and the minimum of upcoming days
dest_sku_arrival_date_movedto1 = (
    AllComb_pd_cpy.sort_values(by="dest_sku_date_arrivaldate", ascending=True)
    .groupby(["Destination", "MaterialID", "Date", "arrivaldate"])
    .agg({"ExcessDemand": "mean", "IntermediateDemandColumn": "mean"})
    .reset_index()
)
dest_sku_arrival_date_movedto1 = dest_sku_arrival_date_movedto1.sort_values(
    by=["Destination", "MaterialID", "arrivaldate", "Date"], ascending=True
)
dest_sku_arrival_date_movedto1["MovableQty1"] = (
    dest_sku_arrival_date_movedto1["ExcessDemand"]
    - dest_sku_arrival_date_movedto1["IntermediateDemandColumn"]
)
dest_sku_arrival_date_movedto1.loc[
    dest_sku_arrival_date_movedto1["MovableQty1"] <= 0, "MovableQty1"
] = 0

dest_sku_arrival_date_movedto2 = (
    AllComb_pd_cpy.sort_values(
        by=["Destination", "MaterialID", "arrivaldate", "Date"], ascending=True
    )
    .groupby(["Destination", "MaterialID", "arrivaldate", "Date"])[column_names]
    .apply(lambda x: x.astype(int).sum())
)  # .reset_index()

dest_sku_arrival_date_movedto2 = (
    dest_sku_arrival_date_movedto2.sort_values(
        by=["Destination", "MaterialID", "arrivaldate", "Date"], ascending=True
    )
    .groupby(["Destination", "MaterialID", "arrivaldate"])[column_names]
    .apply(lambda x: x.cumsum())
)

In [0]:
# Adding constraint in the model
lhs4 = np.concatenate(
    (lhs31, np.array(dest_sku_arrival_date_movedto2[column_names])), axis=0
)
rhs4 = np.concatenate(
    (rhs31, np.array(dest_sku_arrival_date_movedto1["MovableQty1"])), axis=0
)

In [0]:
del dest_sku_arrival_date_movedto1, dest_sku_arrival_date_movedto2, lhs31, rhs31

#Model Output

In [0]:
# shape of the matrix
print(obj.shape, lhs4.shape, rhs4.shape)

In [0]:
# running the solver

lpout = linprog(c=obj, A_ub=lhs4, b_ub=rhs4, method="highs")
lpout

In [0]:
# x1 is solution, that will tell about the qty
x1 = lpout.x
len(x1)
# creating a dataframe using X1
x3 = pd.DataFrame(x1, columns=["QtyMoved"])
# joining the putput with All comb
out = pd.concat([AllComb_pd_cpy, x3], axis=1)

In [0]:
# selecting some columns for the final output
# if there are multiple arrival dates for one lane, earliest arrival date will be selected for the lane
out1 = out[
    ["MaterialID", "Source", "Destination", "SalvageDate", "arrivaldate", "QtyMoved"]
]
STO_qty = spark.createDataFrame(out1)
STO_qty = (
    STO_qty.filter(sf.col("QtyMoved") > 0)
    .groupby("MaterialID", "Source", "Destination", "SalvageDate")
    .agg(
        sf.min("arrivaldate").alias("arrivaldate"), sf.sum("QtyMoved").alias("QtyMoved")
    )
)

In [0]:
# Save the DataFrame in file on ADLS
STO_qty = (
    STO_qty.withColumn("partition_value", sf.lit(partition_path_value))
    .withColumn("Report_Run_Date", sf.lit(ModelRunDate))
    .withColumn("Year", sf.substring("Report_Run_Date", 1, 4))
    .withColumn("Month", sf.substring("Report_Run_Date", 6, 2))
    .withColumn("Day", sf.substring("Report_Run_Date", 9, 2))
)

STO_qty.write.format("delta").mode("append").option("overwriteSchema", "true").option(
    "mergeSchema", "true"
).partitionBy("Year", "Month", "Day").save(destination_path + "/SalvageDateLevelOutput")

In [0]:
dbutils.notebook.exit('continue')